# It's time to know how the datasets are related between them

In [1]:
import pandas as pd
from tqdm import tqdm
import os

In [2]:
# read in the json files
portfolio = pd.read_json('clean_data/portfolio.json')
profile = pd.read_json('clean_data/profile.json')
offer_completed = pd.read_json('clean_data/offers_completed.json')
transactions = pd.read_json('clean_data/transactions.json')
offers_viewed = pd.read_json('clean_data/offers_viewed.json')
offers_received = pd.read_json('clean_data/offers_received.json')

In [3]:
"""
This process takes a while, so I'm gonna store the output dataframe into a file called offers.json.
TODO: Use multiprocessing to make this faster
"""
# Declaring the offers file path
offers_path = 'clean_data/offers.json'

# Checking if the file already exists
if os.path.exists(offers_path):
    # Loading the existing json file
    offers = pd.read_json(offers_path)
else:
    # Defining the important columns from the portfolio dataframe
    portfolio_columns = ['reward', 'difficulty', 'duration', 'bogo', 'discount', 
                         'informational', 'social', 'mobile', 'email', 'web']
    # Declaring a list where all the new rows will be append
    new_rows = []
    print("Starting offers formating.... ({})".format(offers_received.shape[0]))
    
    # Iterating over the received offers
    for index, row in tqdm(offers_received.iterrows()):
        # Getting the important portfolio fields related to the offer
        new_row = portfolio[portfolio.id==row.offer_id].iloc[0][portfolio_columns].to_dict()
        
        # Getting the peron, start_time and due_time
        new_row['person'] = row.person
        new_row['start_time'] = row.time
        new_row['due_time'] = new_row.get('start_time') + (new_row.get('duration')*24)

        # Checking which of the offers were seen
        viewed = offers_viewed.loc[
            (offers_viewed.person==row.person) 
            & (offers_viewed.offer_id==row.offer_id) 
            & (
                (offers_viewed.time>=new_row.get('start_time'))
                | (offers_viewed.time<new_row.get('due_time'))
            )
        ]
        if not viewed.empty:
            new_row['viewed_time'] = viewed.reset_index().iloc[0].time

        # Checking which of the offers were completed
        completed = offer_completed.loc[
            (offer_completed.person==row.person) 
            & (offer_completed.offer_id==row.offer_id) 
            & (
                (offer_completed.time>=new_row.get('start_time'))
                | (offer_completed.time<new_row.get('due_time'))
            )
        ]
        if not completed.empty:
            new_row['completed_time'] = completed.reset_index().iloc[0].time
            
        # Appending the new row obtained
        new_rows.append(new_row)
    
    # Creating a dataframe with the bunch of new rows created
    offers = pd.DataFrame(new_rows)
    
    # Saving the dataframe into a json
    offers.to_json(offers_path)

In [4]:
offers.head()

,reward,difficulty,duration,bogo,discount,informational,social,mobile,email,web,person,start_time,due_time,viewed_time,completed_time
0,5,5,7,1,0,0,0,1,1,1,78afa995795e4d85b5d9ceeca43f5fef,0,168,6.0,132.0
1,5,20,10,0,1,0,0,0,1,1,a03223e636434f42ac4c3df47e8bac43,0,240,6.0,NaN
2,2,10,7,0,1,0,0,1,1,1,e2127556f4f64592b11af22de27a7932,0,168,18.0,NaN
3,2,10,10,0,1,0,1,1,1,1,8ec6ce2a7e7949b1bf142def7d0e0586,0,240,12.0,NaN
4,10,10,5,1,0,0,1,1,1,1,68617ca6246f4fbc85e91a2a49552598,0,120,84.0,NaN


Now It's time to check which of the transactions were influenced by an offer.

In [18]:
offers_influence = transactions.copy()

In [22]:
offers_influence.head()

,person,event,time,amount,informational
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83,0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56,0
12659,54890f68699049c2a04d415abc25e717,transaction,0,13.23,0
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51,0
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97,0


For the informative offers seen, we will think that all purchases made before the expiration date of the offer were influenced by it.

In [21]:
# INFORMATIONAL INFLUENCE
offers_influence['informational'] = 0
informational_offers_viewed = offers[(offers.informational==1) & (~offers.viewed_time.isnull())]
print("Informational offers viewed:", len(informational_offers_viewed))
for index, row in tqdm(informational_offers_viewed.iterrows()):
    offers_influence.loc[(offers_influence.person==row.person) & (offers_influence.time >= row.viewed_time) & (offers_influence.time < row.due_time), 'informational'] = 1

4it [00:00, 34.92it/s]

Informational offers viewed: 11495


11495it [04:36, 41.51it/s]


For the discount offers seen, we will think that if the offer was completed all the purchases made to complete it were influenced by it. If the offer wasn't completed we won't think the purchases were influenced.

In [27]:
# DISCOUNT INFLUENCE
offers_influence['discount'] = 0
discount_offers_viewed_and_completed = offers[(offers.discount==1) & (~offers.viewed_time.isnull()) & (~offers.completed_time.isnull())]
print("Discount offers viewed & completed:", len(discount_offers_viewed_and_completed))
for index, row in tqdm(discount_offers_viewed_and_completed.iterrows()):
    dollars_spent = 0
    transactions_made = offers_influence[(offers_influence.person==row.person) & (offers_influence.time >= row.viewed_time) & (offers_influence.time <= row.completed_time)]
    for idx, transaction_made in transactions_made.iterrows():
        offers_influence.at[idx, 'discount'] = 1
        dollars_spent += transaction_made.amount
        if dollars_spent >= row.difficulty:
            break

6it [00:00, 54.82it/s]

Discount offers viewed & completed: 15846


15846it [04:42, 56.16it/s]


For the BOGO offers we are going to think that the purchases made after the offer was seen or after the first purchase was made (thinking that the users are going to know about their offer after this purchase), were influenced.

In [47]:
# BOGO INFLUENCE
offers_influence['bogo'] = 0
bogo_offers = offers[offers.bogo==1]
print("BOGO offers", len(bogo_offers))
for index, row in tqdm(bogo_offers.iterrows()):
    transactions_made = offers_influence[(offers_influence.person==row.person) & (offers_influence.time >= row.start_time) & ((offers_influence.time.isnull()) | (offers_influence.time <= row.completed_time))]
    first = True
    for idx, transaction_made in transactions_made.iterrows():
        if first is True and transaction_made.time < row.viewed_time:
            first = False
            continue
        offers_influence.at[idx, 'bogo'] = 1

14it [00:00, 137.02it/s]

BOGO offers 30499


30499it [03:18, 153.64it/s]


In [51]:
offers_influence[offers_influence.bogo==1].bogo.count()

15437

In [52]:
offers_influence[offers_influence.discount==1].discount.count()

17613

In [53]:
offers_influence[offers_influence.informational==1].informational.count()

13509

In [54]:
offers_influence.to_json('clean_data/offers_influence.json')